In [1]:
import pandas as pd
import os

In [2]:
current_path = os.getcwd()
print(current_path)

c:\Users\sepujas\Dev\mat\notebooks


In [3]:
base_path = os.path.abspath(os.path.join(current_path, os.pardir))
print(base_path)

c:\Users\sepujas\Dev\mat


In [4]:
vl06i_file_path = os.path.join(base_path, 'data', 'raw', 'tbl_vl06i.txt')
print(vl06i_file_path)
vl06i_preprocessed_path = os.path.join(base_path, 'data', 'preprocessed', 'tbl_vl06i.csv')
print(vl06i_preprocessed_path)

c:\Users\sepujas\Dev\mat\data\raw\tbl_vl06i.txt
c:\Users\sepujas\Dev\mat\data\preprocessed\tbl_vl06i.csv


In [5]:
df_vl06i = pd.read_csv(vl06i_file_path, sep='\t', skiprows=3, encoding='latin1')


In [6]:
df_vl06i.head()

,Unnamed: 0,ShPt,Plnt,SLoc,Delivery,Material,Item Description,Dlv.Qty,BUn,Supplier Name,OPS,WM,GM,Deliv.date
0,NaN,8750,8750,1,681152500,1749,Pinza de montaje KNIPEX,6,PC,HILTI,NaN,NaN,A,08.10.2024
1,NaN,3000,3000,1,680979490,1953,Prolongación de broca TE-FY-E60,6,PC,HILTI,NaN,NaN,A,02.10.2024
2,NaN,3000,3000,1,681150410,1953,Prolongación de broca TE-FY-E60,4,PC,HILTI,NaN,NaN,A,06.11.2024
3,NaN,8650,8650,1,681047863,1954,Prolongación de broca TE-FY-E85,30,PC,HILTI,NaN,NaN,A,30.09.2024
4,NaN,3000,3000,1,681150418,1954,Prolongación de broca TE-FY-E85,4,PC,HILTI,NaN,NaN,A,06.11.2024


In [7]:
unnamed_columns = [col for col in df_vl06i.columns if 'Unnamed:' in col]
df_vl06i.drop(columns=unnamed_columns, inplace=True)
column_titles = df_vl06i.columns.tolist()
new_column_titles = {col: col.strip().replace(' ', '_').replace('-','_').replace('.','') for col in column_titles}
df_vl06i.rename(columns=new_column_titles, inplace=True)

In [8]:
column_counts = df_vl06i.columns.value_counts()
duplicate_columns = column_counts[column_counts > 1].index

In [9]:
for col in duplicate_columns:
    col_indices = [i for i, x in enumerate(df_vl06i.columns) if x == col]
    for j, index in enumerate(col_indices):
        df_vl06i.columns.values[index] = f"{col}_{j+1}"

In [10]:
df_vl06i.columns.tolist()

['ShPt',
 'Plnt',
 'SLoc',
 'Delivery',
 'Material',
 'Item_Description',
 'DlvQty',
 'BUn',
 'Supplier_Name',
 'OPS',
 'WM',
 'GM',
 'Delivdate']

In [11]:
df_vl06i.dtypes

ShPt                  int64
Plnt                  int64
SLoc                  int64
Delivery              int64
Material              int64
Item_Description     object
DlvQty               object
BUn                  object
Supplier_Name        object
OPS                 float64
WM                  float64
GM                   object
Delivdate            object
dtype: object

In [12]:
def transform_columns(df):
    str_columns = [
        'Item_Description', 'BUn', 'Supplier_Name']
    for col in str_columns:
        df[col] = df[col].astype(str).str.strip()

    numeric_columns = ['ShPt', 'Plnt', 'SLoc', 'Material', 'Delivery', 'WM', 'OPS']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int).astype(str)

    df['DlvQty'] = df['DlvQty'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)

    date_columns = ['Delivdate']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", format='%d.%m.%Y')

transform_columns(df_vl06i)

In [13]:
df_vl06i.head()

,ShPt,Plnt,SLoc,Delivery,Material,Item_Description,DlvQty,BUn,Supplier_Name,OPS,WM,GM,Delivdate
0,8750,8750,1,681152500,1749,Pinza de montaje KNIPEX,6.0,PC,HILTI,0,0,A,2024-10-08
1,3000,3000,1,680979490,1953,Prolongación de broca TE-FY-E60,6.0,PC,HILTI,0,0,A,2024-10-02
2,3000,3000,1,681150410,1953,Prolongación de broca TE-FY-E60,4.0,PC,HILTI,0,0,A,2024-11-06
3,8650,8650,1,681047863,1954,Prolongación de broca TE-FY-E85,30.0,PC,HILTI,0,0,A,2024-09-30
4,3000,3000,1,681150418,1954,Prolongación de broca TE-FY-E85,4.0,PC,HILTI,0,0,A,2024-11-06


In [14]:
df_vl06i.dtypes

ShPt                        object
Plnt                        object
SLoc                        object
Delivery                    object
Material                    object
Item_Description            object
DlvQty                     float64
BUn                         object
Supplier_Name               object
OPS                         object
WM                          object
GM                          object
Delivdate           datetime64[ns]
dtype: object

In [15]:
df_vl06i['key_plant'] = df_vl06i['Plnt'] + '/' + df_vl06i['Material']
df_vl06i['key_plant'] = df_vl06i['key_plant'].astype(str).str.strip()
df_vl06i['key_material'] = df_vl06i['ShPt'] + '/' + df_vl06i['Material']
df_vl06i['key_material'] = df_vl06i['key_material'].astype(str).str.strip()

In [16]:
df_vl06i.to_csv(vl06i_preprocessed_path, index=False, sep='|')

In [17]:
df_vl06i.dtypes

ShPt                        object
Plnt                        object
SLoc                        object
Delivery                    object
Material                    object
Item_Description            object
DlvQty                     float64
BUn                         object
Supplier_Name               object
OPS                         object
WM                          object
GM                          object
Delivdate           datetime64[ns]
key_plant                   object
key_material                object
dtype: object